In [49]:
%pip install flatten_json

  Created wheel for flatten-json: filename=flatten_json-0.1.13-py3-none-any.whl size=7967 sha256=aa5076c7a4807f371726208784edd542d382e8bbb0c8965f026aa154fd64b8ca
  Stored in directory: c:\users\apoor\appdata\local\pip\cache\wheels\87\c5\6d\7a772fecd8d6ebae9e60d997f74b9a96ead7d5a0f26a920090
Successfully built flatten-json
Note: you may need to restart the kernel to use updated packages.


In [1]:
import glob
import flatten_json
import json
import pandas as pd
from datetime import datetime

def format_date(num):
    try:
        return datetime.utcfromtimestamp(int(float(num))/1000).strftime('%Y-%m-%d %H:%M:%S')
    except:
        return num

files = glob.glob('data/*.json')

## files[0] & files[3] - simpler JSON files

#read file line by line and append for simpler JSON files
simple_list = [files[0], files[2]]


for file in simple_list:
    temp_list = []
    f = open(file, 'r')
    for line in f:
        temp_list.append(flatten_json.flatten(json.loads(line), '.'))
    df = pd.json_normalize(temp_list)
    for col in df.columns:
        if '$date' in col:
            for i in range(len(df[col])):
                ts = format_date(str(df[col][i]))
                df[col][i] = ts
    df.to_csv(file.replace('.json', '.csv'), index=False)    

<ipython-input-1-3597341ab877>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = ts
C:\Users\apoor\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [27]:
## files[1] is receipts.json - the multi-level JSON file

#read multi-level JSON file
receipt_list = []
for line in open(files[1], 'r'):
    receipt = flatten_json.flatten(json.loads(line))
    receipt_list.append(json.loads(line))

receipt_df = pd.DataFrame.from_dict(pd.json_normalize(receipt_list), orient='columns')

lst_unnest_dct = []
for row in range(len(receipt_df)):
    temp_id = receipt_df.loc[row, '_id.$oid']
    
    lst_rewardsReceiptItemList = receipt_df.loc[row, 'rewardsReceiptItemList']
    try:
        for ele in lst_rewardsReceiptItemList:    
            ele["_id.$oid"] = temp_id
            lst_unnest_dct.append(ele)
    except:
        continue
    
unnested_rewardsReceiptItemList = pd.DataFrame(lst_unnest_dct)

unnest_receipt_df = receipt_df.merge(unnested_rewardsReceiptItemList, on = "_id.$oid", how = 'inner' )

In [30]:
## convert unix timestamp by dividing by 1000ms and store it in all the $date columns

for col in unnest_receipt_df.columns:
    if '$date' in col:
        for i in range(len(unnest_receipt_df[col])):
            ts = format_date(str(unnest_receipt_df[col][i]))
            unnest_receipt_df[col][i] = ts
unnest_receipt_df.to_csv(files[1].replace('.json', '-2.csv'), index=False)

<ipython-input-30-a37e4f549039>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unnest_receipt_df[col][i] = ts


In [ ]:
# The Kimball Approach. 
# The Kimball approach to building the data warehouse starts with identifying the key business processes 
# and the key business questions that the data warehouse needs to answer.

# Business Questions:
# What are the top 5 brands by receipts scanned for most recent month?
# How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
# When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
# Which brand has the most spend among users who were created within the past 6 months?
# Which brand has the most transactions among users who were created within the past 6 months?


# References
# https://www.kimballgroup.com/data-warehouse-business-intelligence-resources/kimball-techniques/dimensional-modeling-techniques/